In [17]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import sys,os
import pickle
from eknot_utils import EventNode,NodeParams,printSummaryOne
from utils import loadPickle,triHits,norm2,printTopWords,coHits,lexRank,norm1
from eknot_utils import *
from time import time


[Xs,vects,DT,ind2obj] = loadPickle('../data/20160207_0213.pickle')
rootParams,rootNodeDescriptor = loadPickle('../output/plsa_20160207_0213_70.pickle')
########### Pz #############
Pz_d = rootNodeDescriptor.Pz_d
Pd = rootNodeDescriptor.Pd
ilist = Pz_d.dot(Pd).argsort()[::-1][2:]
###############

for i in ilist:
    print "############### i ###############",i
    [resDocInd,tweetsObj,tweetsObjDedup,tweetsScore]= loadPickle('../output/summary_20160207_0213_70_'+str(i)+'.pickle')
    

    window = 7
    topK = 100 # ent used
    t0 = time()
    Pw_zs = rootNodeDescriptor.Pw_zs
    Pe_z = Pw_zs[1][:,i]
    evocab = vects[5].get_feature_names()
    vocab = vects[0].get_feature_names()

    ent_ind,ents = getEntInd(evocab,Pe_z,topK)

    print( "entscore in "+str(time() - t0))
    t0 = time()

    newsObj = [ind2obj[n] for n in resDocInd]
    XN,XEn,NEb,sentencesIn,sentencesInObj,ent_text_n = getNewsContext(newsObj,ent_ind,ents,vocab,window)
    print( "get news Context in "+str(time() - t0))
    print len(newsObj),len(sentencesIn),len(set(sentencesIn))
    t0 = time()

    XT,XEt,TEb,tweetsIn,tweetsInObj,ent_text_t = getTweetContext(tweetsObjDedup,ent_ind,ents,vocab,window)
    print( "get tweet Context in "+str(time() - t0))

    print len(tweetsObjDedup),len(tweetsIn),len(set(tweetsIn))
    print XEn.shape,Pe_z.shape


    t0 = time()

    newsScore = XN.dot(Pw_zs[0][:,i])

    tweetsScore = XT.dot(Pw_zs[0][:,i])
   
    print( "init score in "+str(time() - t0))
    t0 = time()

    NE_ = XN.dot(XEn.T) #.multiply(NEb)
    TE_ = XT.dot(XEt.T) #.multiply(TEb)

    ##### weight by pop
    tScore_pop = []
    TE_indptr = TE_.indptr
    TE_data = TE_.data
    for (XTid,t) in enumerate(tweetsInObj):
        st = TE_indptr[XTid]
        en = TE_indptr[XTid+1]
        TE_data[st:en]*=float(t.pop)
        tScore_pop.append(t.pop)
    tScore_pop=np.array(tScore_pop)
    #####

    NE,EN = normBypartite(NE_)
    TE,ET = normBypartite(TE_)
    print( "graph constr in "+str(time() - t0))
    t0 = time()

    ########### params ############
    passes = 100
    wN = 0.7
    wT = 0.7
    ######### tri hits #########
    nScore, tScore = triHits(newsScore, tweetsScore, NE, EN, TE, ET, wN,wT, passes)
    #nScore, tScore = triHits(np.ones(newsScore.shape), np.ones(tweetsScore.shape), NE, EN, TE, ET, 1, 1, 50)
    print np.linalg.norm(norm2(newsScore)[:10] - nScore[:10])
    #print "========="
    #kSummary=5
    #printSummary(norm2(newsScore),norm2(tweetsScore),sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)
    #print "*****"
    #printSummary(nScore,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)

    ######## tri_n ################
    entsScore = XEn.dot(Pw_zs[0][:,i])
    #printRankedEnt(entsScore,ents,100,DISPLAY=True)
    nScore_tri_n, eScore = coHits(newsScore, entsScore, NE, EN, wN, wT, passes)
    #printSummary(nScore_tri_n,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,5)
    #########################

    ############ lexrank ####################
    NN = normalize(XN.dot(XN.T),axis=1,norm='l1')
    nScore_lex = lexRank(newsScore,NN,passes=200)
    #printSummary(nScore_lex,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,10)
    ####################################

    #############co-hits#######################
    NT = normalize(XN.dot(XT.T),axis=1,norm='l1')
    TN = normalize(XT.dot(XN.T),axis=1,norm='l1')
    nScore_co,tScore_co = coHits(newsScore,tweetsScore,NT,TN,wN,wT,passes)
    #printSummary(nScore_co,tScore_co,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,10)
    #########################################

    ################# rank ents ####################
    _res1 = printRankedEnt(NE.T.dot(newsScore),ents,topK)
    res1 = printRankedEnt(NE.T.dot(nScore),ents,topK)
    res2 = printRankedEnt(NE.T.dot(nScore_lex),ents,topK)
    res3 = printRankedEnt(NE.T.dot(nScore_co),ents,topK)
    res4 = printRankedEnt(NE.T.dot(nScore_tri_n),ents,topK)

    # _res2 = printRankedEnt(TE.T.dot(tweetsScore),ents,100)
    # res2 = printRankedEnt(TE.T.dot(tScore),ents,100)

    for e in ents[:20]:
        #print res1[e],_res1[e],res2[e],_res2[e],'>|<',res1[e]-_res1[e],res2[e]-_res2[e], res2[e] - res1[e],e
        print res1[e] - res2[e],res3[e] - res2[e],res1[e]-res4[e],res3[e]-res4[e],\
            '\t|',res1[e],res2[e],res3[e],res4[e],_res1[e],\
            '\t|',res1[e]-_res1[e],res2[e]-_res1[e],res3[e]-_res1[e],res4[e]-_res1[e],\
            '\t|',res2[e] - res4[e],e

    print "##############"        

    #printRankedEnt(NE.T.dot(nScore),ents,100,DISPLAY=True)

    kSummary = int(np.log(len(set(sentencesIn))))
    print kSummary
    b=0.25
    outfile = open('/home/jwang112/Dropbox/linux_buffer/summary_user_study/pair_'+str(i)+'.txt','w')
    outfile.write('########### lex rank ###############\n')
    
    #printSummary(nScore_lex,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)
    printSummaryOne(nScore_lex,sentencesIn,sentencesInObj,XN,kSummary,b,outfile=outfile)

    outfile.write('########### tri graph ##############\n')
    #printSummary(nScore,tScore,sentencesIn,sentencesInObj,tweetsIn,tweetsInObj,kSummary)
    printSummaryOne(nScore,sentencesIn,sentencesInObj,XN,kSummary,b,outfile=outfile)
    outfile.write('###########   tweets  ##############\n')
    printSummaryOne(tScore,tweetsIn,tweetsInObj,XT,2*kSummary,b,outfile=outfile)
    outfile.write('############## most popular tweets ##############\n')
    printSummaryOne(tScore_pop,tweetsIn,tweetsInObj,XT,2*kSummary,b,outfile=outfile)
    outfile.close()
    
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
############### i ############### 50
entscore in 0.0272710323334
get news Context in 1.50938391685
43 1474 1467
append in 1.93048191071
vectorize in 9.56016111374
ents append + vec in 1.00817990303
get tweet Context in 12.524805069
15576 15564 15564
(100, 15042) (4947,)
init score in 0.00123286247253
graph constr in 0.137489080429
==t converge 5 3.47773667512e-10
0.119877799349
==n converge 9 2.80121802644e-10
==n converge 105 9.56740593984e-10
==e converge 6 3.52195901355e-10
0 0 0 0 	| 0 0 0 0 0 	| 0 0 0 0 	| 0 hillary_rodham_clinton_per_|
1 -1 0 -2 	| -3 -4 -5 -3 -4 	| 1 0 -1 1 	| -1 democratic_party_(united_states)_org_|
0 0 0 0 	| 1 1 1 1 1 	| 0 0 0 0 	| 0 bill_clinton_per_|
-2 -1 0 1 	| -3 -1 -2 -3 -3 	| 0 2 1 0 	| 2 new_hampshire_primary_other_|
-1 -1 0 0 	| -5 -4 -5 -5 -5 	| 0 1 0 0 	| 1 barack_obama_per_|
1 -1 0 -2 	| -5 -6 -7 -5 -7 	| 2 1 0 2 	| -1 south_carolina_loc_|
-1 -3 0 -2 	| -8 -7 

In [18]:
ilist

array([50, 60, 29, 16,  2, 15,  0,  9, 68, 67, 44, 11,  7, 14,  5, 17,  4,
       36, 38, 59, 35, 28, 40, 18, 57, 46,  3, 22, 55, 24, 20, 32,  6, 58,
       25, 34,  8, 45, 30, 43, 31, 41, 56, 26, 39, 62, 61, 63, 53, 52, 27,
       66, 47, 42, 33, 54, 23, 19,  1, 21, 69, 51, 13, 10, 48, 65, 12, 49,
       64])

In [16]:
t_c = {}
t_c_dedup = {}
n_c = {}
for i in range(70):
    print "############### i ###############",i
    [resDocInd,tweetsObj,tweetsObjDedup,tweetsScore]= loadPickle('../output/summary_20160207_0213_70_'+str(i)+'.pickle')
    Ntweets,NtweetsDedup,Nnews = len(tweetsObj),len(tweetsObjDedup),len(resDocInd)
    t_c[i] = Ntweets
    t_c_dedup[i] = NtweetsDedup
    n_c[i]=Nnews
    

############### i ############### 0
############### i ############### 1
############### i ############### 2
############### i ############### 3
############### i ############### 4
############### i ############### 5
############### i ############### 6
############### i ############### 7
############### i ############### 8
############### i ############### 9
############### i ############### 10
############### i ############### 11
############### i ############### 12
############### i ############### 13
############### i ############### 14
############### i ############### 15
############### i ############### 16
############### i ############### 17
############### i ############### 18
############### i ############### 19
############### i ############### 20
############### i ############### 21
############### i ############### 22
############### i ############### 23
############### i ############### 24
############### i ############### 25
############### i ############### 26
###########

In [31]:
k=18
t_c[k],np.log(t_c[k]),np.log(5000)

(8466, 9.0438134210808698, 8.5171931914162382)